In [681]:
#Creating Multiple Linear Reg for prediting "Rating" - Cereal dataset
# Importing necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [682]:
#Reading the CSV file and preprocessing
cereal_file = pd.read_csv("cereal.csv",sep = ";")
cereal_file = cereal_file.drop(0)

In [683]:
cereal_file.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
1,100% Bran,N,C,70,4,1,130,10,5,6,280,25,3,1,0.33,68.402973
2,100% Natural Bran,Q,C,120,3,5,15,2,8,8,135,0,3,1,1,33.983679
3,All-Bran,K,C,70,4,1,260,9,7,5,320,25,3,1,0.33,59.425505
4,All-Bran with Extra Fiber,K,C,50,4,0,140,14,8,0,330,25,3,1,0.5,93.704912
5,Almond Delight,R,C,110,2,2,200,1,14,8,-1,25,3,1,0.75,34.384843


In [684]:
type = ['type']
def binary_map(x):
    return x.map({'H': 1, "C": 0})
cereal_file[type] = cereal_file[type].apply(binary_map)

In [685]:
#replacing text entries with dummies

mfr_file = pd.get_dummies(cereal_file['mfr'])
cereal_file = pd.concat([cereal_file, mfr_file], axis = 1)
cereal_file.drop(['mfr'],axis = 1,inplace=True)
cereal_file.drop(['name'],axis = 1,inplace=True)
cereal_file

,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,...,weight,cups,rating,A,G,K,N,P,Q,R
1,0,70,4,1,130,10,5,6,280,25,...,1,0.33,68.402973,0,0,0,1,0,0,0
2,0,120,3,5,15,2,8,8,135,0,...,1,1,33.983679,0,0,0,0,0,1,0
3,0,70,4,1,260,9,7,5,320,25,...,1,0.33,59.425505,0,0,1,0,0,0,0
4,0,50,4,0,140,14,8,0,330,25,...,1,0.5,93.704912,0,0,1,0,0,0,0
5,0,110,2,2,200,1,14,8,-1,25,...,1,0.75,34.384843,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,0,110,2,1,250,0,21,3,60,25,...,1,0.75,39.106174,0,1,0,0,0,0,0
74,0,110,1,1,140,0,13,12,25,25,...,1,1,27.753301,0,1,0,0,0,0,0
75,0,100,3,1,230,3,17,3,115,25,...,1,0.67,49.787445,0,0,0,0,0,0,1
76,0,100,3,1,200,3,17,3,110,25,...,1,1,51.592193,0,1,0,0,0,0,0


In [686]:
#categorizing the data for training and test
np.random.seed(0)
df_train, df_test = train_test_split(cereal_file, train_size = 0.8, test_size = 0.2, random_state = 50)
scaler = MinMaxScaler()
num_vars = ['type','calories','protein','fat','sodium','fiber','carbo','sugars','potass','vitamins','shelf','weight','cups','rating','A','G','K','N','P','Q','R']
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])

In [687]:
#trained data into X and Y
y_train = df_train.pop('rating')
X_train = df_train

In [688]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
scores = []
model = LinearRegression()
model.fit(X_train, y_train)
score = model.score(X_train, y_train)
scores.append(score)
print('\n\nThe Score for multiple linear regression model is : ',score)



The Score for multiple linear regression model is :  0.9999999999999997


In [689]:
X_train = df_train[['calories','protein','fat','sodium','fiber','carbo','sugars','potass','vitamins','cups']]

In [690]:
#trained data modeling
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train)
lr_1 = sm.OLS(y_train.astype(float), X_train_lm.astype(float)).fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [691]:
# scaling
num_vars = ['type','calories','protein','fat','sodium','fiber','carbo','sugars','potass','vitamins','shelf','weight','cups','rating','A','G','K','N','P','Q','R']
df_test[num_vars] = scaler.transform(df_test[num_vars])

In [692]:
#test data as X and Y
y_test = df_test.pop('rating')
X_test = df_test[['calories','protein','fat','sodium','fiber','carbo','sugars','potass','vitamins','cups']]

In [693]:
X_test_m1 = sm.add_constant(X_test)
y_pred_m1 = lr_1.predict(X_test_m1)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


***Write a python function using the numpy library functions that finds the optimal Beta vector (coefficients) that minimizes the residual sum of squares in linear regression***

In [694]:
#calculating beta values 
def estimate_coef_multiple_linear_regression(X, y):
    beta = np.dot((np.linalg.inv(np.dot(X.T,X))), np.dot(X.T,y))
    return beta

In [695]:
# calling the function 
beta = estimate_coef_multiple_linear_regression(X_train, y_train)
beta

array([-0.5290668 ,  0.3536661 ,  0.06236449, -0.23897748,  0.87662653,
        0.73653164,  0.07749175, -0.23343537, -0.1065257 ,  0.11443061])

***Calculate the same linear regression model using the function you developed at step 2 and compare the results with the numbers from step 1 ***



In [696]:
# predicting on the test data with the new coefficient
def manual_predict(X_test, beta):
    return np.dot(X_test, beta)

In [697]:
#value prediction with test data
manual_predictions = manual_predict(X_test, beta)
manual_predictions

array([0.28943389, 0.21969121, 0.75717229, 0.3794705 , 0.36020892,
       0.64179902, 0.14511618, 0.42435126, 0.5723039 , 0.03780099,
       0.41037953, 0.29342804, 0.33414718, 0.31045615, 0.41354987,
       0.26108304])

In [698]:
def res_sum_square(y_hat, y):
    rss = np.sum(np.square((y_hat- y)))
    return rss
    
manual_rss = res_sum_square(y_test,manual_predictions)
reported_rss = res_sum_square(y_test,y_pred_m1)
print(manual_rss)
print(reported_rss)

0.03548503946771031
3.703017431551833e-16


***Write a function to calculate the r-squared: def r_squared(y_hat, y)***

In [699]:
def r_squa(y_hat, y):
    rss = np.sum(np.square((y_hat- y)))
    mean = np.mean(y_hat)
    sst = np.sum(np.square(y_hat-mean))
    r_square = 1 - (rss/sst)
    return r_square

***Use the function in step 4 to calculate the R-squared of the model you created at step 3 and compare the number to the R-squared reported by the python statsmodel or sklearn***

In [700]:
manual_model = r_squa(y_test,manual_predictions)
reported_model = r_squa(y_test,y_pred_m1)
print(manual_model)
print(reported_model)

0.9255325183857698
0.9999999999999992
